In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
from time import sleep
from datetime import datetime
from splinter import Browser
from selenium.webdriver.common.keys import Keys
import time
import json

In [2]:
# Path to chromedriver.exe
executable_path = {'executable_path':'C:/Users/zacha/AppData/Local/bin/chromedriver.exe'}

# Open the Splinter Browswer
browser = Browser('chrome', **executable_path)

In [3]:
# URL to find information on Coronavirus
url = "https://txdshs.maps.arcgis.com/apps/opsdashboard/index.html#/ed483ecd702b4298ab01e8b9cafc8b83"

url3 = "http://publichealth.harriscountytx.gov/Resources/2019-Novel-Coronavirus/Harris-County-COVID-19-Confirmed-Cases"

url4 = "https://www.khou.com/section/news/health/coronavirus"

In [4]:
# Visiting the url 
browser.visit(url)

In [5]:
time.sleep(7)# Creating the Beautiful soup for URL
html = browser.html
soup = bs(html, 'html.parser')

In [6]:
# All the Texas counties
counties = soup.find('nav', attrs={'class':'feature-list'})

# getting all the spans with the information for each Texas county
rows = counties.find_all('span')




In [7]:
counter = 0

county = []
cases = []

for row in rows:
    try:
        print(rows[counter].text.strip().split('\n'))
        county.append(rows[counter].text.strip().split('\n')[0])
        cases.append(rows[counter].text.strip().split('\n')[1])
        counter = counter + 3
        
    except:
        print('failure')

['Harris County', '445']
['Dallas County', '439']
['Travis County', '179']
['Denton County', '148']
['Bexar County', '140']
['Tarrant County', '139']
['Collin County', '128']
['Fort Bend County', '101']
['Montgomery County', '64']
['Brazoria County', '61']
['Galveston County', '49']
['Lubbock County', '41']
['Brazos County', '40']
['Williamson County', '37']
['McLennan County', '36']
['El Paso County', '35']
['Smith County', '30']
['Bell County', '28']
['Hidalgo County', '27']
['Wichita County', '26']
['Webb County', '25']
['Nueces County', '21']
['Cameron County', '20']
['Jefferson County', '16']
['Ellis County', '13']
['Hays County', '13']
['Matagorda County', '12']
['Midland County', '11']
['Guadalupe County', '10']
['Comal County', '9']
['Hardin County', '9']
['Castro County', '8']
['Johnson County', '8']
['Hockley County', '7']
['Kendall County', '7']
['Taylor County', '7']
['Wharton County', '6']
['Bastrop County', '5']
['Val Verde County', '5']
['Washington County', '5']
['Gregg

In [8]:
df = pd.DataFrame(list(zip(county, cases)), columns =['Texas_County', 'COVID_19_Cases'])
df

,Texas_County,COVID_19_Cases
0,Harris County,445
1,Dallas County,439
2,Travis County,179
3,Denton County,148
4,Bexar County,140
...,...,...
113,Shelby County,1
114,Swisher County,1
115,Upshur County,1
116,Van Zandt County,1


In [9]:
# Droping the word 'County'
df['Texas_County'] = df['Texas_County'].apply(lambda x : x.split(' County')[0])

In [10]:
# Revise? 
reported_cases = soup.find_all('text')[6].text
reported_cases

'2,552'

In [11]:
deaths = soup.find_all('text')[8].text
deaths

'34'

In [12]:
df_stats = pd.DataFrame({'Cases':[reported_cases], 'Deaths':[deaths]})
df_stats.to_csv('static/csv/stats.csv', index=False)

In [13]:
browser.visit(url3)

In [14]:
harris_table = browser.find_by_id('covidtable')

In [15]:
html_harris = browser.html
harris_soup = bs(html_harris, 'html.parser')

In [16]:
harris_data = []
table = harris_soup.find('table', attrs={'id':'covidtable'})
table_body = table.find('tbody')

rows = table_body.find_all('tr')
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    harris_data.append([ele for ele in cols if ele]) # Get rid of empty values

In [17]:
columns = ['Patient', 'Sex', 'Age_Range', 'Location','Exposure', 'Status']


In [18]:
harris_data = harris_data[1:]

In [19]:
harris_df = pd.DataFrame(harris_data, columns=columns)

In [20]:
harris_df.sort_values(by=['Age_Range'], inplace=True)

In [21]:
harris_df

,Patient,Sex,Age_Range,Location,Exposure,Status
172,173,Female,0 - 9,NW,Possible Exposure to Case,Active
43,44,Male,0 - 9,NW,Possible Exposure to Case,Active
45,46,Female,0 - 9,NW,Possible Exposure to Case,Active
185,186,Female,0 - 9,SE,Possible Exposure to Case,Active
169,170,Male,10 - 19,NW,Possible Exposure to Case,Active
...,...,...,...,...,...,...
158,159,Female,80+,NW,History of Travel,Active
15,16,Male,80+,NW,No Known Exposure,Deceased
204,205,Male,80+,SE,Investigation Pending,Active
193,194,Female,80+,SE,No Known Exposure,Active


In [22]:
harris_df.to_csv('static/csv/harris_stats.csv', index=False, header=True)

In [23]:
harris_df.head()

,Patient,Sex,Age_Range,Location,Exposure,Status
172,173,Female,0 - 9,NW,Possible Exposure to Case,Active
43,44,Male,0 - 9,NW,Possible Exposure to Case,Active
45,46,Female,0 - 9,NW,Possible Exposure to Case,Active
185,186,Female,0 - 9,SE,Possible Exposure to Case,Active
169,170,Male,10 - 19,NW,Possible Exposure to Case,Active


In [24]:
All_Tex_Counties_df = pd.read_csv('static/csv/Texas_Counties.csv')
All_Tex_Counties_df.head()

,X (Lat),Y (Long),CNTY_NM,CNTY_NBR,FIPS,Shape_Leng,Shape_Area,County Location
0,-97.492799,29.456415,Gonzales,90,48177,2.124911,0.257805,"(-97.492799, 29.456415)"
1,-98.697292,27.043405,Jim Hogg,125,48247,2.271751,0.267624,"(-98.697292, 27.043405)"
2,-97.681378,26.924094,Kenedy,66,48261,5.067864,0.389397,"(-97.681378, 26.924094)"
3,-96.965687,30.310651,Lee,144,48287,2.213853,0.153990,"(-96.965687, 30.310651)"
4,-95.853568,32.211881,Henderson,108,48213,3.152909,0.235056,"(-95.853568, 32.211881)"


In [25]:
All_Tex_Counties = All_Tex_Counties_df['CNTY_NM'].tolist()

In [26]:
df_counties = df['Texas_County'].tolist()

In [27]:
Counties_with_Cases = []

for i in df_counties:
    
    if i in All_Tex_Counties:
        Counties_with_Cases.append(i)
        print(f'{i}---found')
        
    elif i == 'DeWitt':
        Counties_with_Cases.append('De Witt')
        print(f'{i}---found')
    else:
        print(f'Houston we have a problem: {i}')

Harris---found
Dallas---found
Travis---found
Denton---found
Bexar---found
Tarrant---found
Collin---found
Fort Bend---found
Montgomery---found
Brazoria---found
Galveston---found
Lubbock---found
Brazos---found
Williamson---found
McLennan---found
El Paso---found
Smith---found
Bell---found
Hidalgo---found
Wichita---found
Webb---found
Nueces---found
Cameron---found
Jefferson---found
Ellis---found
Hays---found
Matagorda---found
Midland---found
Guadalupe---found
Comal---found
Hardin---found
Castro---found
Johnson---found
Hockley---found
Kendall---found
Taylor---found
Wharton---found
Bastrop---found
Val Verde---found
Washington---found
Gregg---found
Parker---found
Potter---found
Rockwall---found
Angelina---found
Bowie---found
Brown---found
Calhoun---found
Chambers---found
De Witt---found
Eastland---found
Grayson---found
Lamar---found
Medina---found
Randall---found
Rusk---found
Terry---found
Tom Green---found
Victoria---found
Walker---found
Waller---found
Atascosa---found
Austin---found
Cass---

In [28]:
All_Tex_Counties_df = All_Tex_Counties_df.set_index('CNTY_NM')

In [29]:
Geo_df = All_Tex_Counties_df.loc[Counties_with_Cases]
Geo_df.head()

,X (Lat),Y (Long),CNTY_NBR,FIPS,Shape_Leng,Shape_Area,County Location
CNTY_NM,,,,,,,
Harris,-95.397821,29.859671,102,48201,7.731110,0.422798,"(-95.397821, 29.859671)"
Dallas,-96.777819,32.766537,57,48113,1.907782,0.226421,"(-96.777819, 32.766537)"
Travis,-97.781947,30.334233,227,48453,2.458272,0.248996,"(-97.781947, 30.334233)"
Denton,-97.116812,33.205574,61,48121,1.969324,0.238359,"(-97.116812, 33.205574)"
Bexar,-98.519663,29.448845,15,48029,2.729791,0.302846,"(-98.519663, 29.448845)"


In [30]:
Geo_df = Geo_df.reset_index()
Geo_df = Geo_df.rename(columns={"CNTY_NM": "Texas_County"})
Geo_df

,Texas_County,X (Lat),Y (Long),CNTY_NBR,FIPS,Shape_Leng,Shape_Area,County Location
0,Harris,-95.397821,29.859671,102,48201,7.731110,0.422798,"(-95.397821, 29.859671)"
1,Dallas,-96.777819,32.766537,57,48113,1.907782,0.226421,"(-96.777819, 32.766537)"
2,Travis,-97.781947,30.334233,227,48453,2.458272,0.248996,"(-97.781947, 30.334233)"
3,Denton,-97.116812,33.205574,61,48121,1.969324,0.238359,"(-97.116812, 33.205574)"
4,Bexar,-98.519663,29.448845,15,48029,2.729791,0.302846,"(-98.519663, 29.448845)"
...,...,...,...,...,...,...,...,...
113,Shelby,-94.144953,31.792422,210,48419,2.678790,0.205816,"(-94.144953, 31.792422)"
114,Swisher,-101.734951,34.530650,219,48437,1.924577,0.229376,"(-101.734951, 34.53065)"
115,Upshur,-94.941320,32.736283,230,48459,1.799823,0.147655,"(-94.94132, 32.736283)"
116,Van Zandt,-95.836486,32.563690,234,48467,2.402844,0.213777,"(-95.836486, 32.56369)"


In [31]:
result = pd.merge(df, Geo_df, on='Texas_County')
result = result.rename(columns={'X (Lat)':'long', 'Y (Long)':'lat'})

result.head()

,Texas_County,COVID_19_Cases,long,lat,CNTY_NBR,FIPS,Shape_Leng,Shape_Area,County Location
0,Harris,445,-95.397821,29.859671,102,48201,7.731110,0.422798,"(-95.397821, 29.859671)"
1,Dallas,439,-96.777819,32.766537,57,48113,1.907782,0.226421,"(-96.777819, 32.766537)"
2,Travis,179,-97.781947,30.334233,227,48453,2.458272,0.248996,"(-97.781947, 30.334233)"
3,Denton,148,-97.116812,33.205574,61,48121,1.969324,0.238359,"(-97.116812, 33.205574)"
4,Bexar,140,-98.519663,29.448845,15,48029,2.729791,0.302846,"(-98.519663, 29.448845)"


In [32]:
result = result[['Texas_County', 'COVID_19_Cases', 'lat', 'long']]
result

,Texas_County,COVID_19_Cases,lat,long
0,Harris,445,29.859671,-95.397821
1,Dallas,439,32.766537,-96.777819
2,Travis,179,30.334233,-97.781947
3,Denton,148,33.205574,-97.116812
4,Bexar,140,29.448845,-98.519663
...,...,...,...,...
113,Shelby,1,31.792422,-94.144953
114,Swisher,1,34.530650,-101.734951
115,Upshur,1,32.736283,-94.941320
116,Van Zandt,1,32.563690,-95.836486


In [33]:
def df_to_geojson(df, properties, lat='lat', lon='long'):
    prefix = 'var' + ' data' + ' = '
    geojson = {'type':'FeatureCollection', 'features':[]}
    for _, row in df.iterrows():
        feature = {'type':'Feature',
                   'properties':{},
                   'geometry':{'type':'Point','coordinates':[]}}
        feature['geometry']['coordinates'] = [row[lat],row[lon]]
        for prop in properties:
            feature['properties'][prop] = row[prop]
        geojson['features'].append(feature)
    return geojson

cols = ['Texas_County', 'COVID_19_Cases']
javascript_file = df_to_geojson(result, cols)
# {'type': 'FeatureCollection', 'features': [{'geometry': {'type': 'Point', 'coordinates': [-70.5867, -29.9953, 760]}, 'type': 'Feature', 'properties': {'name': 'A', 'description': 'Place a'}}, {'geometry': {'type': 'Point', 'coordinates': [-70.4933, -30.1217, 1250]}, 'type': 'Feature', 'properties': {'name': 'B', 'description': 'Place b'}}, {'geometry': {'type': 'Point', 'coordinates': [-70.5008, -30.0953, 1185]}, 'type': 'Feature', 'properties': {'name': 'C', 'description': 'Place c'}}]}

In [34]:

with open('static/js/map.json','w') as f:
    json.dump(javascript_file, f, indent=4, sort_keys=False)

In [35]:
browser.visit(url4)

In [36]:
headlines_harris = browser.html
headline_soup = bs(headlines_harris, 'html.parser')

In [37]:
# lis = headline_soup.find_all('li', class_="headline-list-with-abstract__item")
links = headline_soup.find_all('div', class_="headline-list-with-abstract__headline")

titles = []
hrefs = []

In [38]:
recent = headline_soup.find_all('div', class_="story-snapshot-with-abstract__headline")[0]

In [39]:
recent

<div class="story-snapshot-with-abstract__headline">
<a class="story-snapshot-with-abstract__link" data-tracking="story-snapshot-with-abstract/headlineLinkClicked" href="/article/news/health/coronavirus/coronavirus-live-updates-march-29/285-e387e28f-f52e-40c4-b68e-c48f1b3c5b0e">
<div class="story-snapshot-with-abstract__desktop-title dot-ellipsis dot-height-180" style="overflow-wrap: break-word;">
                        Watch live: White House Coronavirus Task Force providing update
                    </div>
<div class="story-snapshot-with-abstract__mobile-title dot-ellipsis dot-height-135" style="overflow-wrap: break-word;">
                        Watch live: White House Coronavirus Task Force providing update
                    </div>
</a>
</div>

In [40]:
recent_title = recent.find('div', class_="story-snapshot-with-abstract__desktop-title dot-ellipsis dot-height-180").text.strip()
recent_title

'Watch live: White House Coronavirus Task Force providing update'

In [41]:
titles.append(recent_title)
hrefs.append('https://www.khou.com' + recent.a['href'])

In [42]:
links

[<div class="headline-list-with-abstract__headline">
 <a class="headline-list-with-abstract__link" data-tracking="headline-list-with-abstract/headlineLinkClicked" href="/article/news/health/coronavirus/coronavirus-updates-houston-live-log-march-29/285-a63d46dc-c130-48b7-bdbd-2c84660a8e92">
                     Coronavirus updates: Gov. Abbott expands orders requiring certain travelers to self-quarantine upon arriving in Texas
                 </a>
 </div>, <div class="headline-list-with-abstract__headline">
 <a class="headline-list-with-abstract__link" data-tracking="headline-list-with-abstract/headlineLinkClicked" href="/article/news/health/coronavirus/country-legend-joe-diffie-61-dies-from-covid-19-complications/285-416c7c6a-454a-4f7c-87b5-dac7537c18a9">
                     Country legend Joe Diffie, 61, dies from COVID-19 complications
                 </a>
 </div>, <div class="headline-list-with-abstract__headline">
 <a class="headline-list-with-abstract__link" data-tracking="head

In [43]:

for article in links:
    
    try:
        
        title = article.find('a', class_="headline-list-with-abstract__link").text
        
        link = article.a['href']
        
        if (title and link):
            
            
            print('-------------')
            print(title)
            print(link)
            titles.append(title.strip())
            hrefs.append('https://www.khou.com' + link)
            
    except:
        print('no')

-------------

                    Coronavirus updates: Gov. Abbott expands orders requiring certain travelers to self-quarantine upon arriving in Texas
                
/article/news/health/coronavirus/coronavirus-updates-houston-live-log-march-29/285-a63d46dc-c130-48b7-bdbd-2c84660a8e92
-------------

                    Country legend Joe Diffie, 61, dies from COVID-19 complications
                
/article/news/health/coronavirus/country-legend-joe-diffie-61-dies-from-covid-19-complications/285-416c7c6a-454a-4f7c-87b5-dac7537c18a9
-------------

                    Here's how Houston-area residents can get tested for coronavirus
                
/article/news/health/coronavirus/houston-coronavirus-testing/285-c95ce8c4-f876-4d6c-8645-4922578617df
-------------

                    Map: Keeping track of Houston-area coronavirus cases
                
/article/news/health/coronavirus/houston-area-coronavirus-cases/285-192b8879-08b6-4ec0-8a0f-a0487ead1453
-------------

              

In [44]:
latest_headlines_df = pd.DataFrame(list(zip(titles, hrefs)), columns =['Headline', 'link'])

In [45]:
latest_headlines_df

,Headline,link
0,Watch live: White House Coronavirus Task Force...,https://www.khou.com/article/news/health/coron...
1,Coronavirus updates: Gov. Abbott expands order...,https://www.khou.com/article/news/health/coron...
2,"Country legend Joe Diffie, 61, dies from COVID...",https://www.khou.com/article/news/health/coron...
3,Here's how Houston-area residents can get test...,https://www.khou.com/article/news/health/coron...
4,Map: Keeping track of Houston-area coronavirus...,https://www.khou.com/article/news/health/coron...
5,Harris County Jail inmate tests positive for c...,https://www.khou.com/article/news/health/coron...
6,"Maria Mercader, CBS News journalist for three ...",https://www.khou.com/article/news/health/coron...
7,Brazos County confirms first COVID-19 case; no...,https://www.khou.com/article/news/community/br...
8,FEMA extends flood insurance renewal policies ...,https://www.khou.com/article/news/health/coron...
9,German finance official's death linked to viru...,https://www.khou.com/article/news/health/coron...


In [46]:
latest_headlines_df.to_csv('static/csv/headlines.csv', index=False, header=True)

In [47]:
browser.quit()